# Video Game Dataset

Analysis on the video game dataset to gain business insights. 

## Exploring the Data

In [ ]:
import pandas as pd
import numpy as np

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder, KBinsDiscretizer
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

# vgsales = pd.read_csv('/content/vgsales_v5.csv')
vgsales = pd.read_excel("vgsales_v5.xlsx")

In [ ]:
vgsales.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Years_In_Market,Avg_Annual_Sales,Year_1_Sales,First_Party,Platform_Type
0,1,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,10,8.274000,53.7810,Nintendo,Console
1,2,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,31,1.298065,26.1560,Nintendo,Console
2,3,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82,8,4.477500,23.2830,Nintendo,Console
3,4,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00,7,4.714286,21.4500,Nintendo,Console
4,5,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,20,1.568500,20.3905,Nintendo,HH


In [ ]:
vgsales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15979 entries, 0 to 15978
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Rank              15979 non-null  int64  
 1   Name              15979 non-null  object 
 2   Platform          15979 non-null  object 
 3   Year              15979 non-null  int64  
 4   Genre             15979 non-null  object 
 5   Publisher         15945 non-null  object 
 6   NA_Sales          15979 non-null  float64
 7   EU_Sales          15979 non-null  float64
 8   JP_Sales          15979 non-null  float64
 9   Other_Sales       15979 non-null  float64
 10  Global_Sales      15979 non-null  float64
 11  Years_In_Market   15979 non-null  int64  
 12  Avg_Annual_Sales  15979 non-null  float64
 13  Year_1_Sales      15979 non-null  float64
 14  First_Party       15979 non-null  object 
 15  Platform_Type     15979 non-null  object 
dtypes: float64(7), int64(3), object(6)
memor

In [ ]:
vgsales.describe()

,Rank,Year,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Years_In_Market,Avg_Annual_Sales,Year_1_Sales
count,15979.000000,15979.000000,15979.00000,15979.000000,15979.000000,15979.000000,15979.000000,15979.000000,15979.000000,15979.000000
mean,7991.070593,2006.197071,0.26976,0.149093,0.079514,0.048892,0.547537,9.802929,0.087326,0.355899
std,4613.123753,5.714810,0.82956,0.512557,0.314536,0.191612,1.580275,5.714810,0.323913,1.027179
min,1.000000,1980.000000,0.00000,0.000000,0.000000,0.000000,0.010000,1.000000,0.000417,0.006500
25%,3996.500000,2003.000000,0.00000,0.000000,0.000000,0.000000,0.060000,6.000000,0.007500,0.039000
50%,7991.000000,2007.000000,0.08000,0.020000,0.000000,0.010000,0.180000,9.000000,0.020909,0.117000
75%,11985.500000,2010.000000,0.24000,0.110000,0.040000,0.040000,0.490000,13.000000,0.060000,0.318500
max,15981.000000,2015.000000,41.49000,29.020000,10.220000,10.570000,82.740000,36.000000,14.240000,53.781000


In [ ]:
vgsales.isnull().any()

Rank                False
Name                False
Platform            False
Year                False
Genre               False
Publisher            True
NA_Sales            False
EU_Sales            False
JP_Sales            False
Other_Sales         False
Global_Sales        False
Years_In_Market     False
Avg_Annual_Sales    False
Year_1_Sales        False
First_Party         False
Platform_Type       False
dtype: bool

### Analysis of Global Sales with other attributes
Here the Global Sales is plotted on the y axis, with the other attributes on the x axis. It can be seen clearly observed that:
* North american and europe sales are similar.
* Year 1 sales has a direct relationship with Global Sales (they are identical).
* Japan is the most different market.  

The pie chart shows that:
* Almost 50% of sales come from North America - so that is the biggest market.
* 27.2% of sales come from Europe.

From the bar plots it is clear that:
* Platform, Genre and Platform Type are good indicators of what is popular globally.
* In platform type - Console is a huge percentage of the total.
* The top 3-5 publishers make up 99% of the total sales. 

In [ ]:
fig = make_subplots(rows=3, cols=2, start_cell="top-left", 
                    subplot_titles=("NA_Sales vs Global Sales", "EU_Sales vs Global Sales", "JP_Sales vs Global Sales", "Other_Sales vs Global_Sales", "Avg_Annual_Sales vs Global_Sales", "Year_1_Sales vs Global_Sales"))

fig.add_trace(go.Scatter(x=vgsales["NA_Sales"], y=vgsales["Global_Sales"], mode="markers"),
              row=1, col=1)

fig.add_trace(go.Scatter(x=vgsales["EU_Sales"], y=vgsales["Global_Sales"], mode="markers"),
              row=1, col=2)

fig.add_trace(go.Scatter(x=vgsales["JP_Sales"], y=vgsales["Global_Sales"], mode="markers"),
              row=2, col=1)

fig.add_trace(go.Scatter(x=vgsales["Other_Sales"], y=vgsales["Global_Sales"], mode="markers"),
              row=2, col=2)

fig.add_trace(go.Scatter(x=vgsales["Avg_Annual_Sales"], y=vgsales["Global_Sales"], mode="markers"),
              row=3, col=1)

fig.add_trace(go.Scatter(x=vgsales["Year_1_Sales"], y=vgsales["Global_Sales"], mode="markers"),
              row=3, col=2)

fig.show()

In [ ]:
df = vgsales[["NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales"]]
df = df.sum().reset_index()
fig = px.pie(df, values=0, names='index')
fig.show()

In [ ]:
for label in ["Platform", "Genre", "Publisher", "Platform_Type"]:
    df = vgsales[[label, "Global_Sales"]]
    df = df.groupby(label).sum().reset_index().sort_values('Global_Sales', ascending=False)
    fig = px.bar(df, x=label, y="Global_Sales")
    fig.show()

# Linear Regression Models

## Model 1
Predicting global sales using data from EU and JP sales only. This would give a good idea about how the global market performs. Reduces the risk, as initial testing data can be used to predict North American sales which makes up 50% of the total sales.  

Independent Variables: NA_Sales, Year_1_Sales  
Dependent Variable: Global_Sales


In [ ]:
# Note: adding year 1 sales leads to 100% accuracy which we dont want

y = vgsales["Global_Sales"]

features = ["EU_Sales", "JP_Sales"]
X = vgsales[features]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

regressor = LinearRegression()  
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

# Using the mean-squared error
error = metrics.mean_squared_error(y_test, y_pred)
print("Error:", error)

In [ ]:
fig = go.Figure()
# fig.add_trace(go.Scatter(x=y_test.index, y=y_test, mode="markers", name="Actual"))
fig.add_trace(go.Scatter(x=y_test, y=y_pred, mode="markers", name="Predicted"))
fig.show()

# Ideal case, the graph would be a perfectly straight line. 

## Model 2

Predicting global sales using platform type and Europe sales only. This is useful because 25% of the total sales (EU_Sales) can be used give a good prediction of the total sales all around the world. Great to launch a new product and test it out in a smaller market. 

Independent Variables: Platform_Type, EU_Sales
Dependent Variable: Global_Sales

In [ ]:
# Linear Regression 2

y = vgsales["Global_Sales"]
# One-hot encoding Platform_Type - since this is categorical 
X = pd.concat([pd.get_dummies(vgsales["Platform_Type"]), vgsales["EU_Sales"]], axis=1, sort=False)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
regressor = LinearRegression()  
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

error = metrics.mean_squared_error(y_test, y_pred)
print("Error:", error)

In [ ]:
fig = go.Figure()
# fig.add_trace(go.Scatter(x=y_test.index, y=y_test, mode="markers", name="Actual"))
fig.add_trace(go.Scatter(x=y_test, y=y_pred, mode="markers", name="Predicted"))
fig.show()

#KNN Models

## Model 1
Using the Genre and Years in market, we predict the platform type. This gives an insight into what platforms have been in the market for the longest, and also helps to analyze the relationship between Genre and Platform Type. This can be useful when releasing a game on a new platform or maybe innovating platform types.   

Features: Genre, Years_In_Market  
Predicting: Platform_Type

In [ ]:
fig = px.scatter(vgsales, x="Genre", y="Global_Sales", color="Platform_Type")
fig.show()

In [ ]:
X = pd.concat([pd.get_dummies(vgsales["Genre"]), vgsales["Years_In_Market"]], axis=1, sort=False)

mmsc = MinMaxScaler()
X["Years_In_Market"] = mmsc.fit_transform(X["Years_In_Market"].values.reshape(-1, 1))

y = vgsales["Platform_Type"]
le = LabelEncoder()
y = le.fit_transform(y.astype('str'))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

knn = KNeighborsClassifier(n_neighbors=13)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

## Model 2
Given a Genre and Platform we try to predict how well this will do on global sales. This can be used to infer relationships like Action Games on a Console do much better than Action Games on PC.
  
Features: Genre, Platform  
Predicting: Global Sales bracket (discretized to two bins)

In [ ]:
X = pd.concat([pd.get_dummies(vgsales["Genre"]), pd.get_dummies(vgsales["Platform"])], axis=1, sort=False)
mmsc = MinMaxScaler()

y = vgsales["Global_Sales"]
disc = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy="quantile")
y = disc.fit_transform(y.values.reshape(-1, 1))
y = np.ravel(y, order='C')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

knn = KNeighborsClassifier(n_neighbors=15)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.2909887359198999


# Decision trees
## Model 1
Given the Platform, Platform_Type, Publisher and First_Party, we try to predict what Genre it belongs to.  

The low accuracy in this case maybe because:
* Genre is a createive choice of the game designers.
* Every platform will have a collection of games that belong to all Genres to attract a bigger market. This it does not have any correlation.

In [ ]:
# This tells us that genre is independent and has no relation with others
# Maybe because all platforms try to appleal to gamers who like all genre

X = vgsales[["Platform", "Publisher", "Platform_Type", "First_Party"]]
X["Platform"] = le.fit_transform(vgsales["Platform"].astype("str"))
X["Platform_Type"] = le.fit_transform(vgsales["Platform_Type"].astype("str"))
X["Publisher"] = le.fit_transform(X["Publisher"].astype("str"))
X["First_Party"] = le.fit_transform(X["First_Party"].astype("str"))

y = vgsales["Genre"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

clf = DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

## Model 2. 
For a given Platform, Platform Type, Genre and Sales form North America, we try to predict the global sales bracker (discretized). This can be useful in analyzing what combinations of these features have a higher chance on success in a global market. 

In [ ]:
X = vgsales[["Platform", "Platform_Type", "Genre", "NA_Sales"]]
X["Platform"] = le.fit_transform(vgsales["Platform"].astype("str"))
X["Platform_Type"] = le.fit_transform(vgsales["Platform_Type"].astype("str"))
X["Genre"] = le.fit_transform(X["Genre"].astype("str"))

y = vgsales["Global_Sales"]
# change n_bins to 2 for 89% acc
disc = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy="quantile")
y = disc.fit_transform(y.values.reshape(-1, 1))
y = np.ravel(y, order='C')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

clf = DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

print("Accuracy:", metrics.accuracy_score(y_test, y_pred))